# TPUs Attempt at running  Effnet on this massive dataset

uses the Ian Pan Jpeg datset:

https://www.kaggle.com/vaillant/rsna-str-pe-detection-jpeg-256

and some tricks from this Pytorch notebook:

https://www.kaggle.com/orkatz2/pulmonary-embolism-pytorch-train

as well as some standard Pytorch from the tutorials page:

https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html#load-data

With TPU support from this Notebook:

https://www.kaggle.com/tanulsingh077/pytorch-xla-understanding-tpu-s-and-xla

## Results so far

* got it running on one core but still takes 55mins per epoch!
* now parallelising to run on the 8 cores available.

In [ ]:
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version nightly --apt-packages libomp5 libopenblas-dev

In [ ]:
from sklearn import metrics
from sklearn.model_selection import train_test_split
import transformers
from transformers import AdamW

import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla
import torch_xla.debug.metrics as met

import torch_xla.utils.utils as xu

import torch_xla.test.test_utils as test_utils

import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install -U git+git://github.com/lilohuang/PyTurboJPEG.git

In [ ]:
from turbojpeg import TurboJPEG, TJPF_GRAY, TJSAMP_GRAY


In [ ]:
!conda install -c conda-forge gdcm -y

In [ ]:
!pip install torch_optimizer

In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import cv2
import os
from matplotlib import pyplot as plt
from torch.utils.data import TensorDataset, DataLoader,Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2
from skimage.color import gray2rgb
import functools
import torch
from torch import Tensor
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.nn.parameter import Parameter
from torch.optim import lr_scheduler
import torch_optimizer as optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler
from tqdm.auto import tqdm
from matplotlib import animation, rc
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import pydicom as dcm
import gdcm
import time

In [ ]:
#torch.backends.cudnn.benchmark = True

In [ ]:
train_path = '../input/rsna-str-pulmonary-embolism-detection/train.csv'
test_path = pd.read_csv("../input/rsna-str-pulmonary-embolism-detection/test.csv")
jpeg_path = '../input/rsna-str-pe-detection-jpeg-256/train-jpegs'
files = glob.glob('../input/rsna-str-pulmonary-embolism-detection/train/*/*/*.dcm')

In [ ]:
train_df=pd.read_csv(train_path)

In [ ]:
list_of_jpgs= [i for i in range(1,10)]

In [ ]:
columns_only_for_info = ["qa_motion","qa_contrast","flow_artifact","true_filling_defect_not_pe"]

Can take the onlyforinfo cols out as they are not needed.

In [ ]:
train_df.iloc[list_of_jpgs,:]

In [ ]:
target_columns = ['StudyInstanceUID','SeriesInstanceUID','SOPInstanceUID','pe_present_on_image', 'negative_exam_for_pe', 'rv_lv_ratio_gte_1', 
                  'rv_lv_ratio_lt_1','leftsided_pe', 'chronic_pe','rightsided_pe', 
                  'acute_and_chronic_pe', 'central_pe', 'indeterminate']

In [ ]:
train_df[target_columns]

In [ ]:
class jpg_dataset(Dataset):
    def __init__(self, root_dir, df,  transforms = None):
        
        super().__init__()
        self.df = df[target_columns]
        self.root_dir = root_dir
        self.transforms = transforms

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, ndx):
        row = self.df.iloc[ndx,:]
        in_file = open(glob.glob(f"{root_dir}/{row[0]}/{row[1]}/*{row[2]}.jpg")[0], 'rb')
        img = jpeg.decode(in_file.read())
        in_file.close()
        label = row[3:].astype(int)
        label[2:] = label[2:] if label[0]==1 else 0
        #img /= 255.0
        
        #Retriving class label

        
        #Applying transforms on image
        if self.transforms:
            img = self.transforms(image=img)['image']
        

        return (img,label.values)

In [ ]:
root_dir=jpeg_path
jpeg = TurboJPEG()

## Trial of dataset class

## Real Dataloaders

In [ ]:
StudyInstanceUID = list(set(train_df['StudyInstanceUID']))
print(len(StudyInstanceUID))
t_df = train_df[train_df['StudyInstanceUID'].isin(StudyInstanceUID[0:500])]
v_df = train_df[train_df['StudyInstanceUID'].isin(StudyInstanceUID[500:600])]

In [ ]:
t_df.head()

In [ ]:
import os
os.environ['XLA_USE_BF16']="1"
os.environ['XLA_TENSOR_ALLOCATOR_MAXSIZE'] = '100000000'

In [ ]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': A.Compose(
    [
        #A.SmallestMaxSize(max_size=160),
        #A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
        A.RandomCrop(height=128, width=128),
        #A.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ]
),
    'val': A.Compose(
    [
        A.SmallestMaxSize(max_size=160),
        A.CenterCrop(height=128, width=128),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ]
),
}



## Build model

In [ ]:
!pip install efficientnet_pytorch

In [ ]:
from efficientnet_pytorch import EfficientNet


In [ ]:
class EfficientNetEncoderHead(nn.Module):
    def __init__(self, depth, num_classes):
        super(EfficientNetEncoderHead, self).__init__()
        self.depth = depth
        self.base = EfficientNet.from_pretrained(f'efficientnet-b{self.depth}')
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.output_filter = self.base._fc.in_features
        self.classifier = nn.Linear(self.output_filter, num_classes)
    def forward(self, x):
        x = self.base.extract_features(x)
        x = self.avg_pool(x).squeeze(-1).squeeze(-1)
        x = self.classifier(x)
        return x

In [ ]:
model = EfficientNetEncoderHead(depth=0, num_classes=10)

In [ ]:
def train_model(model, no_epochs=25):
    since = time.time()
    criterion = torch.nn.BCEWithLogitsLoss()
    batch_size = 16
    no_epochs = 4
    num_workers = 0
    lr = 1e-3
    device = xm.xla_device()
    model = model.to(device)
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    # Not sure if creating datasets and dataloaders in the train fn will cause OOM
    #print("Initializing Datasets and Dataloaders...")

    # Create training and validation datasets
    image_datasets = {}
    image_datasets['train'] = jpg_dataset(root_dir=jpeg_path,df=t_df, transforms = data_transforms['train'])
    image_datasets['val'] = jpg_dataset(root_dir=jpeg_path,df=v_df, transforms = data_transforms['val'])
    dataloaders_dict={}
    # Create training and validation dataloaders
    train_sampler = None
    if xm.xrt_world_size() > 1:
        #print ('more than one core found')
        train_sampler = torch.utils.data.distributed.DistributedSampler(
              image_datasets['train'],
              num_replicas=xm.xrt_world_size(),
              rank=xm.get_ordinal(),
              shuffle=True)
    dataloaders_dict['train'] = torch.utils.data.DataLoader(
                image_datasets['train'],
                batch_size=batch_size,
                sampler=train_sampler,
                shuffle=False if train_sampler else True,
                num_workers=num_workers)
    dataloaders_dict['val'] = torch.utils.data.DataLoader(
            image_datasets['val'],
            batch_size=batch_size,
            shuffle=False,
            num_workers=num_workers)
    dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
    device_loader={}
    para_loader={}
    #device_loader['train'] = pl.MpDeviceLoader(dataloaders_dict['train'], device)
    #device_loader['val'] = pl.MpDeviceLoader(dataloaders_dict['val'], device)
    lr = lr * xm.xrt_world_size()
    optimizer = radam(model.parameters(), lr=lr, betas=(0.9,0.999), eps=1e-3, weight_decay=1e-4)
    scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(dataloaders_dict['train'])*no_epochs, eta_min=1e-6) # dataloader should be train one - change later

    # Decay LR by a factor of 0.1 every 7 epochs
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
        
    for epoch in range(no_epochs):
        para_loader['train'] = pl.ParallelLoader(dataloaders_dict['train'], [device]).per_device_loader(device)
        para_loader['val'] = pl.ParallelLoader(dataloaders_dict['val'], [device]).per_device_loader(device)

        xm.master_print('Epoch {}/{}'.format(epoch, no_epochs - 1))
        xm.master_print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            current_loss_mean = 0.0
            running_corrects = 0

            # Iterate over data.
            tqdm_loader = tqdm(para_loader[phase])

            for batch_idx, (inputs,labels) in enumerate(tqdm_loader):
                inputs = inputs.to(device).float()
                labels = labels.to(device).float()
                
                #inputs, labels = inputs.cuda().float(), labels.cuda().float() 

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history only if in train
                with torch.set_grad_enabled(phase == 'train'):

                    outputs = model(inputs)
                    loss = criterion(outputs.float(), labels.float())

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        #if batch_idx % 10 == 0:
                        #    xm.master_print(f'batch_idx={batch_idx} of length {len(para_loader[phase])}, loss={loss}')
                        #optimizer.step()
                        xm.optimizer_step(optimizer)

                # statistics
                running_loss += loss.item() * inputs.size(0)
                current_loss_mean = (current_loss_mean * batch_idx + loss) / (batch_idx + 1)
                if batch_idx % 10 == 0:
                    xm.master_print(f'batch_idx={batch_idx} of length {len(para_loader[phase])}, loss={current_loss_mean}')
                running_corrects += torch.sum(outputs == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            score = 1-current_loss_mean
            xm.master_print('metric {}'.format(score))
            
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            xm.master_print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    xm.master_print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    xm.master_print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    xm.save(model.state_dict(),f'model{k}.bin')
    return model

In [ ]:
def radam(parameters, lr=1e-3, betas=(0.9, 0.999), eps=1e-8, weight_decay=0):
    if isinstance(betas, str):
        betas = eval(betas)
    return optim.RAdam(parameters,
                      lr=lr,
                      betas=betas,
                      eps=eps,
                      weight_decay=weight_decay)

In [ ]:
import copy

In [ ]:
flags={}

In [ ]:
#train_model(model, criterion, optimizer, scheduler, no_epochs=no_epochs)

In [ ]:
def _mp_fn(rank, flags):
    global acc_list
    torch.set_default_tensor_type('torch.FloatTensor')
    a = train_model(model=model, no_epochs=3)

FLAGS={}
xmp.spawn(_mp_fn, args=(FLAGS,), nprocs=8, start_method='fork')